In [ ]:
import os
import cv2
import numpy as np

# Set dataset path (this must match your Kaggle dataset name)
data_path = '/kaggle/input/betel-leaf/Comprehensive Betel Leaf Disease Dataset for Advanced Pathology Research/Betel Leaf Dataset/Betel Leaf Dataset/Betel Leaf Dataset/Augmented_Dataset'  # <- Replace with actual dataset name

categories = sorted(os.listdir(data_path))
print("Number of classes:", categories)
noofClasses = len(categories)
print("Total number of classes:", noofClasses)
print("Importing images...")

labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

print("Label dictionary:", label_dict)
print("Categories:", categories)
print("Labels:", labels)

img_size = 124
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)

    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Direct grayscale

        try:
            imga = cv2.resize(img, (img_size, img_size))  # Resize

            # Z-score normalization
            mean = np.mean(imga)
            std = np.std(imga)
            imga = (imga - mean) / (std if std != 0 else 1)

            data.append(imga)
            target.append(label_dict[category])
        except Exception as e:
            print(f'Exception processing image {img_name}: {e}')

data = np.array(data)
target = np.array(target)

print("Data shape:", data.shape)
print("Target shape:", target.shape)

# Save arrays to Kaggle's writable directory
np.save("/kaggle/working/FirstStageX.npy", data)
np.save("/kaggle/working/FirstStagey.npy", target)

print(" Files saved to /kaggle/working/")


In [20]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Paths
original_data_dir = '/kaggle/input/betel-leaf/Comprehensive Betel Leaf Disease Dataset for Advanced Pathology Research/Betel Leaf Dataset/Betel Leaf Dataset/Betel Leaf Dataset/Augmented_Dataset'
base_output_dir = '/kaggle/working/split_data'
train_dir = os.path.join(base_output_dir, 'train')
val_dir = os.path.join(base_output_dir, 'val')
test_dir = os.path.join(base_output_dir, 'test')

# Create dirs
for d in [train_dir, val_dir, test_dir]:
    os.makedirs(d, exist_ok=True)

# Split each class folder into train/val/test
classes = os.listdir(original_data_dir)

for cls in classes:
    cls_dir = os.path.join(original_data_dir, cls)
    images = os.listdir(cls_dir)

    # Shuffle and split
    train_imgs, test_imgs = train_test_split(images, test_size=0.10, random_state=42)
    train_imgs, val_imgs = train_test_split(train_imgs, test_size=0.1111, random_state=42)  # 10/90 = 0.1111 for validation (0.10 total)

    for img_name, subset in zip(
        [train_imgs, val_imgs, test_imgs],
        [train_dir, val_dir, test_dir]
    ):
        subset_cls_dir = os.path.join(subset, cls)
        os.makedirs(subset_cls_dir, exist_ok=True)

        for img in img_name:
            src = os.path.join(cls_dir, img)
            dst = os.path.join(subset_cls_dir, img)
            shutil.copy(src, dst)

print("✅ Dataset split into train/val/test with 80:10:10 ratio.")

✅ Dataset split into train/val/test with 80:10:10 ratio.


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

# --- CBAM attention blocks ---
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_dense_one = tf.keras.layers.Dense(channel // ratio, activation='relu')
    shared_dense_two = tf.keras.layers.Dense(channel)
    avg_pool = tf.keras.layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = shared_dense_one(avg_pool)
    avg_pool = shared_dense_two(avg_pool)
    max_pool = tf.keras.layers.GlobalMaxPooling2D()(input_feature)
    max_pool = shared_dense_one(max_pool)
    max_pool = shared_dense_two(max_pool)
    cbam_feature = tf.keras.layers.Add()([avg_pool, max_pool])
    cbam_feature = tf.keras.layers.Activation('sigmoid')(cbam_feature)
    cbam_feature = tf.keras.layers.Reshape((1, 1, channel))(cbam_feature)
    return tf.keras.layers.Multiply()([input_feature, cbam_feature])

def spatial_attention(input_feature):
    avg_pool = tf.keras.layers.Lambda(
        lambda x: tf.reduce_mean(x, axis=3, keepdims=True),
        output_shape=lambda input_shape: (input_shape[0], input_shape[1], input_shape[2], 1)
    )(input_feature)
    max_pool = tf.keras.layers.Lambda(
        lambda x: tf.reduce_max(x, axis=3, keepdims=True),
        output_shape=lambda input_shape: (input_shape[0], input_shape[1], input_shape[2], 1)
    )(input_feature)
    concat = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])
    cbam_feature = tf.keras.layers.Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')(concat)
    return tf.keras.layers.Multiply()([input_feature, cbam_feature])

def cbam_block(input_feature):
    x = channel_attention(input_feature)
    x = spatial_attention(x)
    return x

# --- Model building ---
def build_cbam_cnn(input_shape=(224, 224, 3), num_classes=3):
    inputs = tf.keras.layers.Input(shape=input_shape)

    x = tf.keras.layers.Conv2D(32, 7, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(32, 7, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = cbam_block(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(64, 5, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(64, 5, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = cbam_block(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(128, 3, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(128, 3, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = cbam_block(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(256, 3, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(256, 3, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = cbam_block(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    return tf.keras.models.Model(inputs, outputs)

# Load model weights
model_path = '/kaggle/input/mine/tensorflow2/default/1/betel_leaf_model (1).h5'
model = build_cbam_cnn()
model.load_weights(model_path)

# Assume test_data is your test data generator (without shuffle)
X_test = []
y_test = []
for i in range(len(test_data)):
    x_batch, y_batch = test_data[i]
    X_test.append(x_batch)
    y_test.append(y_batch)
X_test = np.vstack(X_test)
y_test = np.argmax(np.vstack(y_test), axis=1)

# Preprocess input for model
from tensorflow.keras.applications.resnet50 import preprocess_input
X_test_preprocessed = preprocess_input(X_test.copy())

# Helper function to prepare image for display
def prepare_for_display(img):
    """
    Convert any image tensor to uint8 RGB for display:
    - If float image (likely normalized), scale to [0,255]
    - If grayscale (1 channel), repeat to 3 channels
    - Clip and convert to uint8
    """
    if img.dtype != np.uint8:
        img_min = np.min(img)
        img_max = np.max(img)
        img = (img - img_min) / (img_max - img_min + 1e-8)  # normalize 0-1
        img = (img * 255).astype(np.uint8)
    if img.shape[-1] == 1:
        img = np.repeat(img, 3, axis=-1)
    return img

# Find last Conv2D layer
last_conv_layer_name = None
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
assert last_conv_layer_name is not None, "No Conv2D layer found in model."

print(f"Last Conv2D layer for Grad-CAM: {last_conv_layer_name}")

# Grad-CAM function
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Overlay heatmap on image
def overlay_heatmap(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, colormap)
    if img.dtype != np.uint8:
        img = prepare_for_display(img)
    if img.shape[-1] == 1:
        img = np.repeat(img, 3, axis=-1)
    if img.shape != heatmap_color.shape:
        heatmap_color = cv2.resize(heatmap_color, (img.shape[1], img.shape[0]))
    superimposed_img = cv2.addWeighted(heatmap_color, alpha, img, 1 - alpha, 0)
    return superimposed_img

# Predict on test set
preds = model.predict(X_test_preprocessed, verbose=0)
y_pred = np.argmax(preds, axis=1)
num_classes = model.output_shape[-1]

# Get 3 correct samples per class
correct_per_class = {i: [] for i in range(num_classes)}
for i in range(len(y_test)):
    true_label = int(y_test[i])
    pred_label = int(y_pred[i])
    if true_label == pred_label and len(correct_per_class[true_label]) < 20:
        correct_per_class[true_label].append(i)
    if all(len(v) >= 20 for v in correct_per_class.values()):
        break

# Plot results
for class_id, indices in correct_per_class.items():
    for idx in indices:
        original_img = X_test[idx]
        input_img = np.expand_dims(X_test_preprocessed[idx], axis=0)

        heatmap = make_gradcam_heatmap(input_img, model, last_conv_layer_name)
        overlay = overlay_heatmap(original_img, heatmap)

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(prepare_for_display(original_img))
        plt.title(f"Original Image (Class {class_id})")
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(heatmap, cmap='jet')
        plt.title("Grad-CAM Heatmap")
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(overlay[..., ::-1])  # BGR to RGB for cv2 overlay
        plt.title("Overlay")
        plt.axis('off')

        plt.tight_layout()
        plt.show()
